In [ ]:
# TODO:
# 1. Process all games chronologically 
# 2. Fix the issue of a player being missing the mapping JSON. Right now they won't be processed.
# 3. Write team rating into a JSON for storage?

from glob import glob
import json

# Constants
player_id = "player_id"
handle = "handle"

platformGameId = "platformGameId"
participantMapping = "participantMapping"
participantId = "participantId"

gameInfoEventInd = 0
platformGameId = "platformGameId"
eventType = "eventType"
champion_kill = "champion_kill"
killer = "killer"
game_info = "game_info"

STARTING_TEAM_RATING = 1000
MINIMUM_TEAM_RATING = 0 
ENTRY_RATING_FEE = 150 # 'SR' cost to play a game
WIN_MATCH_SR_GAIN = 100
KILL_SR_GAIN = 10
WARD_KILL_SR_GAIN = 1
TOWER_KILL_SR_GAIN = 15
DRAGON_KILL_SR_GAIN = 10
RIFT_KILL_SR_GAIN = 10
BARON_KILL_SR_GAIN = 20

# Global values
playerIDPlayerMap = {}
teamIDTeamMap = {}
mappingDataMap = {}
matchIdTeams = {}

# Helper Functions
# Returns a storage based on read contents of file
def readJson(fileName):
    with open(fileName) as file:
        file_contents = file.read()
    print("Processing file: " + fileName)
    return json.loads(file_contents)

# Helper Objects
# Player object to represent a pro player
class Player():
    def __init__(self, playerInfo):
        # Identification info
        self._playerId = playerInfo[player_id]
        self._handle = playerInfo[handle]

        # Games' specific info
        self._matchCount = 0
        self._totalKills = 0 
    
    # Return average kills per match
    def getAvgKillsPerMatch(self):
        # Note: protects against 0 matches played
        return round(self._totalKills / max(self._matchCount, 1), 2)

    def addToMatchCount(self, toAdd):
        self._matchCount += toAdd
    
    def addToTotalKill(self, toAdd):
        self._totalKills += toAdd

    def getPlayerId(self):
        return self._playerId
    
    def getHandle(self):
        return self._handle
    
    def getMatchCount(self):
        return self._matchCount

    def getTotalKills(self):
        return self._totalKills

# Team represents a pro team
class Team():
    def __init__(self, teamInfo):
        self._teamId = teamInfo['team_id']
        self._name = teamInfo['name']
        self._rating = STARTING_TEAM_RATING
        self._wins = 0
        self._losses = 0
        self._kills = 0
        self._wardKills = 0
        self._towerKills = 0
        self._dragonKills = 0
        self._baronKills = 0

    def getTeamId(self):
        return self._teamId

    def getName(self):
        return self._name
    
    def getRating(self):
        return self._rating

    def addRating(self, addRating):
        self._rating += addRating
    
    def subtractRating(self, subtractRating):
        self._rating -= subtractRating
        # Rating can't go below zero
        self._rating = max(self._rating, MINIMUM_TEAM_RATING)

    def getWins(self):
        return self._wins
    
    def addWin(self):
        self._wins += 1
    
    def getLosses(self):
        return self._losses

    def addLoss(self):
        self._losses += 1

    def getKills(self):
        return self._kills

    def addKills(self, killAddition):
        self._kills += killAddition
    
    def getWardKills(self):
        return self._wardKills

    def addWardKills(self, wardKillAddition):
        self._wardKills += wardKillAddition
    
    def getTowerKills(self):
        return self._towerKills

    def addTowerKills(self, towerKillAddition):
        self._towerKills += towerKillAddition
    
    def getDragonKills(self):
        return self._dragonKills

    def addDragonKills(self, dragonKillAddition):
        self._dragonKills += dragonKillAddition
    
    def getBaronKills(self):
        return self._baronKills

    def addBaronKills(self, baronKillAddition):
        self._baronKills += baronKillAddition


In [ ]:
# Build player metadata
# Map of { player_id : playerObjs }
# Note: Can store all players ever since count of all pro players ever should be under order of 10 thousands

# Read player.json to build this map
playerJson = readJson('esports-data/players.json')
for player in playerJson:
    # TODO: Guard against player w/o id or handle 
    playerIDPlayerMap[player[player_id]] = Player(player)

print("Evaluating across " + str(len(playerIDPlayerMap)) + " players\n")

In [ ]:
# Build team metadata
# Map of { team_id: teamObjs }
# Read player.json to build this map
teamJson = readJson('esports-data/teams.json')
for team in teamJson:
    teamIDTeamMap[team['team_id']] = Team(team)

print("Evaluating across " + str(len(teamIDTeamMap)) + " teams\n")


In [ ]:
# Build the mapping data from JSON of same name
# Map of { platformGameId : { participantId : player_id }}
mappingJson = readJson('esports-data/mapping_data.json')
for mapping in mappingJson:
    # TODO: Guard against participant mapping missing
    currParticipantMapping = mapping[participantMapping]
    mappingDataMap[mapping[platformGameId]] = currParticipantMapping

In [ ]:
# Read tournament JSON to build a map of { match_id: 100: team_id_blue, 200: team_id_red, winner: 100 OR 200 OR 0 }
# 0 means tie
tournamentJson = readJson('esports-data/tournaments.json')
for tournament in tournamentJson:
    if 'stages' in tournament:
        for stage in tournament['stages']:
            if 'sections' in stage:
                for section in stage['sections']:
                    if 'matches' in section:
                        for match in section['matches']:
                            if 'id' in match and 'teams' in match and len(match['teams']) == 2:
                                if 'games' in match:
                                    for game in match['games']:
                                        key = match['id'] + '|game' + str(game['number'])
                                        matchIdTeams[key] = {'winningSide': 0}

                                        # Side for team
                                        for team in game['teams']:
                                            if team['side'] == 'blue':
                                                matchIdTeams[key][100] = team['id']
                                            else:
                                                matchIdTeams[key][200] = team['id']
                                            if 'result' in team and team['result'] and 'outcome' in team['result'] and team['result']['outcome'] == 'win':
                                                matchIdTeams[key]['winningSide'] = 100 if team['side'] == 'blue' else 200

print("Evaluating across " + str(len(matchIdTeams)) + " matches\n")

In [ ]:
def getWinnerLoser(winLossMap):
    if winLossMap['winningSide'] == 100:
        return winLossMap[100], winLossMap[200]
    elif winLossMap['winningSide'] == 200:
        return winLossMap[200], winLossMap[100]
    
    return None, None

# Object to hold information both team's performance for a match
class TeamsInMatch:
    def __init__(self, teamID100, teamID200):
        self._teamID100 = teamID100
        self._teamID200 = teamID200
        self._players100 = set()
        self._players200 = set()
        self._srGain100 = 0
        self._srGain200 = 0
        self._killCount100 = 0
        self._killCount200 = 0
        self._wardKillCount100 = 0
        self._wardKillCount200 = 0
        self._towerKillCount100 = 0
        self._towerKillCount200 = 0
        self._dragonKillCount100 = 0
        self._dragonKillCount200 = 0
        self._baronKillCount100 = 0
        self._baronKillCount200 = 0
    
    def getTeamID(self, sideID):
        if sideID == 100:
            return self._teamID100
        elif sideID == 200:
            return self._teamID200

    def addPlayer(self, sideID, playerID):
        if sideID == 100:
            self._players100.add(playerID)
        elif sideID == 200:
            self._players200.add(playerID)
    
    def addChampionKill(self, playerID):
        if playerID in self._players100:
            self._srGain100 += KILL_SR_GAIN
            self._killCount100 += 1
        elif playerID in self._players200:
            self._srGain200 += KILL_SR_GAIN
            self._killCount200 += 1
        else:
            print(str(playerID) + " not found for match")

    def addWardKill(self, playerID):
        if playerID in self._players100:
            self._srGain100 += WARD_KILL_SR_GAIN
            self._wardKillCount100 += 1
        elif playerID in self._players200:
            self._srGain200 += WARD_KILL_SR_GAIN
            self._wardKillCount200 += 1
        else:
            print(str(playerID) + " not found for match")
    
    def addTowerKill(self, teamID):
        if teamID == 100:
            self._srGain100 += TOWER_KILL_SR_GAIN
            self._towerKillCount100 += 1
        elif teamID == 200:
            self._srGain200 += TOWER_KILL_SR_GAIN
            self._towerKillCount200 += 1
        else:
            print(str(teamID) + " not found for match")
    
    def addDragonKill(self, playerID):
        if playerID in self._players100:
            self._srGain100 += DRAGON_KILL_SR_GAIN
            self._dragonKillCount100 += 1
        elif playerID in self._players200:
            self._srGain200 += DRAGON_KILL_SR_GAIN
            self._dragonKillCount200 += 1
        else:
            print(str(playerID) + " not found for match")

    def addBaronKill(self, playerID):
        if playerID in self._players100:
            self._srGain100 += BARON_KILL_SR_GAIN
            self._baronKillCount100 += 1
        elif playerID in self._players200:
            self._srGain200 += BARON_KILL_SR_GAIN
            self._baronKillCount200 += 1
        else:
            print(str(playerID) + " not found for match")
    
    def getTeamIDSRGain(self, teamSide):
        if teamSide == 100:
            return self._teamID100, self._srGain100
        elif teamSide == 200:
            return self._teamID200, self._srGain200
    
    def getTeamIDKillGain(self, teamSide):
        if teamSide == 100:
            return self._teamID100, self._killCount100
        elif teamSide == 200:
            return self._teamID200, self._killCount200

    def getTeamIDWardKillGain(self, teamSide):
        if teamSide == 100:
            return self._teamID100, self._wardKillCount100
        elif teamSide == 200:
            return self._teamID200, self._wardKillCount200
    
    def getTeamIDTowerKillGain(self, teamSide):
        if teamSide == 100:
            return self._teamID100, self._towerKillCount100
        elif teamSide == 200:
            return self._teamID200, self._towerKillCount200
    
    def getTeamIDDragonKillGain(self, teamSide):
        if teamSide == 100:
            return self._teamID100, self._dragonKillCount100
        elif teamSide == 200:
            return self._teamID200, self._dragonKillCount200
    
    def getTeamIDBaronKillGain(self, teamSide):
        if teamSide == 100:
            return self._teamID100, self._baronKillCount100
        elif teamSide == 200:
            return self._teamID200, self._baronKillCount200


# Read each JSON in ./games by order of game occurance
# PER Game
# Reading participant 
# Using mapping_data.json create a map of { participantID (1 - 10 w.r.t match) : player_id } 
# TODO: Loop through all games in .games/
for f_name in glob('games/*.json'):
    currGameJson = readJson(f_name)

    # TODO: For now, drop a game if first event isn't about the game
    #  Consider altering in the future
    gameInfoEvent = currGameJson[gameInfoEventInd]
    if gameInfoEvent[eventType] != game_info:
        continue

    gameName = gameInfoEvent['gameName']
    if not gameName in matchIdTeams:
        continue

    # Winning and losing team of the match
    winner, loser = getWinnerLoser(matchIdTeams[gameName])
    if winner == None or loser == None:
        continue
    teamIDTeamMap[winner].addWin()
    teamIDTeamMap[loser].addLoss()
    print('Winner: ' + teamIDTeamMap[winner].getName() + " | Loser: " + teamIDTeamMap[loser].getName())

    # Create object to track team performance for the match
    teamInMatch = TeamsInMatch(matchIdTeams[gameName][100], matchIdTeams[gameName][200])

    # A many to two map which maps player ID to team SR objects
    incompletePlayer = False
    for participant in gameInfoEvent['participants']:
        if not 'teamID' in participant or not 'participantID' in participant:
            incompletePlayer = True
            break
        teamInMatch.addPlayer(participant['teamID'], participant['participantID'])
    if incompletePlayer:
        print('Ignoring match, incomplete match info')
        continue

    # SR coming into the game
    orgSR100 = teamIDTeamMap[matchIdTeams[gameName][100]].getRating()
    orgSR200 = teamIDTeamMap[matchIdTeams[gameName][200]].getRating()

    # SR calculations
    # 1. Teams 'pay' entry SR
    teamIDTeamMap[winner].subtractRating(ENTRY_RATING_FEE)
    teamIDTeamMap[winner].addRating(WIN_MATCH_SR_GAIN)
    teamIDTeamMap[loser].subtractRating(ENTRY_RATING_FEE)

    # Determine participant Id to PlayerId mapping via platformGameId
    currPartIdPlayerId = mappingDataMap[gameInfoEvent[platformGameId]]

    # Add one match played to each player
    for participantId in currPartIdPlayerId:
        playerIDPlayerMap[currPartIdPlayerId[participantId]].addToMatchCount(1)

    # Per X kill event in game.
    for event in currGameJson:
        if 'kill' in event[eventType] and 'killer' in event:
            killerId = str(event[killer])
            if killerId in currPartIdPlayerId:
                if event[eventType] == champion_kill: 
                    playerIDPlayerMap[currPartIdPlayerId[killerId]].addToTotalKill(1)
                    teamInMatch.addChampionKill(int(killerId))
                elif event[eventType] == 'ward_killed':
                    teamInMatch.addWardKill(int(killerId))
                elif event[eventType] == 'epic_monster_kill' and 'monsterType' in event and event['monsterType'] == 'dragon':
                    teamInMatch.addDragonKill(int(killerId))
                elif event[eventType] == 'epic_monster_kill' and 'monsterType' in event and event['monsterType'] == 'baron':
                    teamInMatch.addBaronKill(int(killerId))
        elif event[eventType] == 'building_destroyed' and 'buildingType' in event and event['buildingType'] == 'turret' and 'teamID' in event:
            killerId = str(event['teamID'])
            teamInMatch.addTowerKill(int(event['teamID']))

        
    
    # Provide SR gain and kill gain for both teams
    team100Id, team100SRGain = teamInMatch.getTeamIDSRGain(100)
    teamIDTeamMap[team100Id].addRating(team100SRGain)
    _, team100KillGain = teamInMatch.getTeamIDKillGain(100)
    teamIDTeamMap[team100Id].addKills(team100KillGain)
    _, team100WardKillGain = teamInMatch.getTeamIDWardKillGain(100)
    teamIDTeamMap[team100Id].addWardKills(team100WardKillGain)
    _, team100TowerKillGain = teamInMatch.getTeamIDTowerKillGain(100)
    teamIDTeamMap[team100Id].addTowerKills(team100TowerKillGain)
    _, team100DragonKillGain = teamInMatch.getTeamIDDragonKillGain(100)
    teamIDTeamMap[team100Id].addDragonKills(team100DragonKillGain)
    _, team100BaronKillGain = teamInMatch.getTeamIDBaronKillGain(100)
    teamIDTeamMap[team100Id].addBaronKills(team100BaronKillGain)

    team200Id, team200SRGain = teamInMatch.getTeamIDSRGain(200)
    teamIDTeamMap[team200Id].addRating(team200SRGain)
    _, team200KillGain = teamInMatch.getTeamIDKillGain(200)
    teamIDTeamMap[team200Id].addKills(team200KillGain)
    _, team200WardKillGain = teamInMatch.getTeamIDWardKillGain(200)
    teamIDTeamMap[team200Id].addWardKills(team200WardKillGain)
    _, team200TowerKillGain = teamInMatch.getTeamIDTowerKillGain(200)
    teamIDTeamMap[team200Id].addTowerKills(team200TowerKillGain)
    _, team200DragonKillGain = teamInMatch.getTeamIDDragonKillGain(200)
    teamIDTeamMap[team200Id].addDragonKills(team200DragonKillGain)
    _, team200BaronKillGain = teamInMatch.getTeamIDBaronKillGain(200)
    teamIDTeamMap[team200Id].addBaronKills(team200BaronKillGain)
    
    print(teamIDTeamMap[team100Id].getName())
    print('> SR change: ' + str(teamIDTeamMap[team100Id].getRating() - orgSR100) + ' | ' + str(orgSR100) + ' => ' + str(teamIDTeamMap[team100Id].getRating()))
    print('> Kills: ' + str(team100KillGain))
    print('> Ward Kills: ' + str(team100WardKillGain))
    print('> Tower Kills: ' + str(team100TowerKillGain))
    print('> Dragon Kills: ' + str(team100DragonKillGain))
    print('> Baron Kills: ' + str(team100BaronKillGain))
    print(teamIDTeamMap[team200Id].getName())
    print('> SR change: ' + str(teamIDTeamMap[team200Id].getRating() - orgSR200) + ' | ' + str(orgSR200) + ' =>' + str(teamIDTeamMap[team200Id].getRating()))
    print('> Kills: ' + str(team200KillGain))
    print('> Ward Kills: ' + str(team200WardKillGain))
    print('> Tower Kills: ' + str(team200TowerKillGain))
    print('> Dragon Kills: ' + str(team200DragonKillGain))
    print('> Baron Kills: ' + str(team200BaronKillGain))

    print('\n')

In [ ]:
# Print out all the teams by order of SR. This is the ranking of teams
teamIDTeamMap = dict(sorted(teamIDTeamMap.items(), key = lambda item: item[1].getRating(), reverse=True))
itter = 0
for teamID in teamIDTeamMap:
    teamObj = teamIDTeamMap[teamID]
    print(teamObj.getName())
    print('SR:' + str(teamObj.getRating()))
    print('Wins:' + str(teamObj.getWins()))
    print('Lossess:' + str(teamObj.getLosses()))
    print('Champ Kills:' + str(teamObj.getKills()))
    print('Dragon Kills:' + str(teamObj.getDragonKills()))
    print('Baron Kills:' + str(teamObj.getBaronKills()))
    print('Tower Kills:' + str(teamObj.getTowerKills()))
    print('Ward Kills:' + str(teamObj.getWardKills()))
    print('\n')
    itter += 1
    if itter == 25:
        break

In [ ]:
# TODO: Remove
# Prints out average kill of all players that have played at least one game
# for player in playerIDPlayerMap.values():
#     if player.getMatchCount() > 0:
#         print(player.getHandle())
#         print("Avg kill per match: " + str(player.getAvgKillsPerMatch()))
#         print("Totale kills: " + str(player.getTotalKills()))
#         print("Matches played: " + str(player.getMatchCount()))
#         print("\n")